<a href="https://colab.research.google.com/github/Beneroms/ETL-PIPELINE-PROJECT/blob/main/ETL_PIPELINE_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Python Extract Transform Load Example
"""

# %%
import requests
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
def extract()-> dict:
    """ This API extracts data from
    http://universities.hipolabs.com
    """
    API_URL = "http://universities.hipolabs.com/search?country=United+States"
    data = requests.get(API_URL).json()
    return data

In [ ]:
# Make the API request
API_URL = "http://universities.hipolabs.com/search?country=United+States" # Define API_URL
response = requests.get(API_URL)

# Check the Content-Type header
content_type = response.headers['Content-Type']
print(f"Content-Type: {content_type}")  # Print the content type

Content-Type: application/json


In [ ]:
def transform(data:dict) -> pd.DataFrame:
    """ Transforms the dataset into desired structure and filters"""
    df = pd.DataFrame(data)
    print(f"Total Number of universities from API {len(data)}")
    df = df[df["name"].str.contains("California")]
    print(f"Number of universities in california {len(df)}")
    df['domains'] = [','.join(map(str, l)) for l in df['domains']]
    df['web_pages'] = [','.join(map(str, l)) for l in df['web_pages']]
    df = df.reset_index(drop=True)
    return df[["domains","country","web_pages","name"]]

In [ ]:
def load(df:pd.DataFrame)-> None:
    """ Loads data into a sqllite database"""
    disk_engine = create_engine('sqlite:///my_lite_store.db')
    df.to_sql('cal_uni', disk_engine, if_exists='replace')


In [ ]:
# %%
data = extract()
df = transform(data)
load(df)


# %%

Total Number of universities from API 2334
Number of universities in california 44


In [ ]:

data = extract()
df = transform(data)
load(df)



Total Number of universities from API 2334
Number of universities in california 44


In [ ]:
!apt-get install sqlite3


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  sqlite3-doc
The following NEW packages will be installed:
  sqlite3
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 768 kB of archives.
After this operation, 1,873 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 sqlite3 amd64 3.37.2-2ubuntu0.3 [768 kB]
Fetched 768 kB in 0s (2,995 kB/s)
Selecting previously unselected package sqlite3.
(Reading database ... 123597 files and directories currently installed.)
Preparing to unpack .../sqlite3_3.37.2-2ubuntu0.3_amd64.deb ...
Unpacking sqlite3 (3.37.2-2ubuntu0.3) ...
Setting up sqlite3 (3.37.2-2ubuntu0.3) ...
Processing triggers for man-db (2.10.2-1) ...


In [10]:
!sqlite3 --version


3.37.2 2022-01-06 13:25:41 872ba256cbf61d9290b571c0e6d82a20c224ca3ad82971edc46b29818d5dalt1


In [11]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('my_lite_store.db')

# Create a cursor object to interact with the database
cursor = conn.cursor()

# List all tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Print the available tables
print("Tables in the database:", tables)

# Close the connection when done
conn.close()


Tables in the database: [('cal_uni',)]


In [12]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('my_lite_store.db')

# Create a cursor object
cursor = conn.cursor()

# Fetch the first 5 rows from the 'cal_uni' table
cursor.execute("SELECT * FROM cal_uni LIMIT 5;")
rows = cursor.fetchall()

# Print the fetched rows
print("Sample data from the 'cal_uni' table:")
for row in rows:
    print(row)

# Close the connection
conn.close()


Sample data from the 'cal_uni' table:
(0, 'csuci.edu', 'United States', 'https://www.csuci.edu/', 'California State University Channel Islands')
(1, 'calbaptist.edu', 'United States', 'http://www.calbaptist.edu/', 'California Baptist University')
(2, 'calsouthern.edu', 'United States', 'http://www.calsouthern.edu/', 'California Southern University')
(3, 'cpp.edu', 'United States', 'http://www.cpp.edu/', 'California Polytechnic State University, Pomona')
(4, 'calcoast.edu', 'United States', 'http://www.calcoast.edu/', 'California Coast University')


In [13]:
import pandas as pd
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('my_lite_store.db')

# Query the 'cal_uni' table and load it into a pandas DataFrame
df = pd.read_sql_query("SELECT * FROM cal_uni", conn)

# Show the first few rows of the DataFrame
print(df.head())

# Close the connection
conn.close()


   index          domains        country                    web_pages  \
0      0        csuci.edu  United States       https://www.csuci.edu/   
1      1   calbaptist.edu  United States   http://www.calbaptist.edu/   
2      2  calsouthern.edu  United States  http://www.calsouthern.edu/   
3      3          cpp.edu  United States          http://www.cpp.edu/   
4      4     calcoast.edu  United States     http://www.calcoast.edu/   

                                              name  
0      California State University Channel Islands  
1                    California Baptist University  
2                   California Southern University  
3  California Polytechnic State University, Pomona  
4                      California Coast University  


In [14]:
# List the tables in the SQLite database
!sqlite3 my_lite_store.db .tables

# View all records in the 'cal_uni' table
!sqlite3 my_lite_store.db "SELECT * FROM cal_uni LIMIT 5;"


cal_uni
0|csuci.edu|United States|https://www.csuci.edu/|California State University Channel Islands
1|calbaptist.edu|United States|http://www.calbaptist.edu/|California Baptist University
2|calsouthern.edu|United States|http://www.calsouthern.edu/|California Southern University
3|cpp.edu|United States|http://www.cpp.edu/|California Polytechnic State University, Pomona
4|calcoast.edu|United States|http://www.calcoast.edu/|California Coast University
